In [ ]:
import numpy as np

X = torch.tensor(np.loadtxt("design_parameters.dat"), dtype=torch.float32)

N, d = X.shape

r = 1

f = torch.tensor(np.loadtxt("non_dimensionalized_efficiency.dat"), dtype=torch.float32)

m=256 

feature_model = RFFFeatures(r=r, m=m, sigma=1.0, device=X.device, dtype=X.dtype) 
M_star, alpha_star = stochastic_grassmann_ridge( X=X, f=f, r=r, feature_model=feature_model, lam=1e-2, eta=5e-2, batch_size=256, steps=3000, tol=1e-7, patience=30, seed=0, ) 
